**First run the ```init.sh``` file before running this notebook**

## Reading the file

idée : voir l'évolution du réseau avec une animation (à mettre à disposition sur le github?) avec un passage des années.

In [1]:
import pandas as pd
import networkx as nx
import os

#chemin d'accès
directory = os.getcwd()
data_path = directory[:directory.rfind('/')] + '/data'
figure_path = directory[:directory.rfind('/')] + '/figures'
print(f"data_path : {data_path}")
print(f"figure_path : {figure_path}")

data_path : /home/onyxia/work/citations-network/data
figure_path : /home/onyxia/work/citations-network/figures


In [31]:
import polars as pl
import networkx as nx

# Load CSVs with Polars (super fast!)
nodes_df = pl.read_csv("/home/onyxia/work/citations-network/data/raw/nodes/papers.csv", separator='\t', truncate_ragged_lines=True, schema_overrides={"halid": pl.Utf8})
edges_df = pl.read_csv("/home/onyxia/work/citations-network/data/raw/edges/paper__cites__paper.csv",  separator='\t', truncate_ragged_lines=True, schema_overrides={"halid": pl.Utf8})

In [28]:
#print first two rows
with open("/home/onyxia/work/citations-network/data/raw/nodes/papers.csv", 'r') as f:
    for i in range(2):  # Change the range if you want more rows
        line = f.readline()
        print(f"Line {i + 1}: {line.strip()}")


Line 1: halid	year	title	lang	domain	paper_idx
Line 2: 01907786	2018	Le Système Comptable Financier algérien entre les « Full IFRS » et la norme IFRS PME : Etude qualitative de sa mise en oeuvre par les entreprises	fr	['shs.gestion']	0


In [29]:
start_line = 18_662_035
end_line = 18_662_037

with open("/home/onyxia/work/citations-network/data/raw/nodes/papers.csv", 'r') as f:
    for i, line in enumerate(f):
        if i < start_line:
            continue
        if i > end_line:
            break
        print(f"Line {i + 1}: {line.strip()}")


Line 18662036: 2015	Heteroatom-doped carbon nanostructures as oxygen reduction reaction catalysts in acidic media: an overview			565887
Line 18662037: 2011	Electrooxidation of glycerol studied by combined in situ IR spectroscopy and online mass spectrometry under continuous flow conditions			565888
Line 18662038: 2002	Electro-oxidation of glycerol on platinum dispersed in polyaniline matrices			565889


In [40]:
nodes_df['paper_idx']df.filter(df.is_duplicated())


paper_idx
i64
0
1
2
3
4
…
565885
565886
565887


In [34]:
nodes_pd = nodes_df.to_pandas()
edges_pd = edges_df.to_pandas()
G = nx.DiGraph()

In [38]:
# Add nodes with attributes
nodes = nodes_pd.set_index('paper_idx').to_dict(orient='index')
G.add_nodes_from(nodes.items())

# Add edges with attributes
edges = edges_pd.to_dict(orient='records')
G.add_edges_from((e['paper_idx'], e['c_paper_idx'], {k: v for k, v in e.items() if k not in ['paper_idx', 'c_paper_idx']}) for e in edges)



ValueError: DataFrame index must be unique for orient='index'.

In [33]:
nodes_df.drop_nulls()

halid,year,title,lang,domain,paper_idx
str,i64,str,str,str,i64
"""01907786""",2018,"""Le Système Comptable Financier…","""fr""","""['shs.gestion']""",0
"""02029554""",2018,"""Master Droit européen""","""fr""","""[]""",1
"""01724695""",2017,"""Inversion methods for the reco…","""fr""","""['math.math-ap', 'math.math-na…",2
"""01730414""",2017,"""split jacobians and lower boun…","""en""","""['math.math-gm']""",3
"""01735234""",2018,"""Sociotechnique de la presbyaco…","""fr""","""['shs.socio']""",4
…,…,…,…,…,…
"""00203801""",2010,"""Profiting in the info-communic…","""en""","""['shs.eco']""",14715
"""02319006""",2018,"""Évolution de la représentation…","""fr""","""['sdv.mhep']""",14716
"""02401065""",2018,"""Crustal growth and Pan-African…","""fr""","""['sdu.stu.te']""",14717


In [32]:
edges_df

paper_idx,c_paper_idx
i64,i64
0,7705
0,7706
0,7707
0,7721
0,7728
…,…
7630,23412
7669,291146
7669,291147


In [30]:
null_count = nodes_df["domain"].is_null().sum()
print(f"Number of nulls in 'halid': {null_count}")

Number of nulls in 'halid': 18171080


In [5]:
dir(nodes_df)

['__add__',
 '__annotations__',
 '__array__',
 '__arrow_c_stream__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_accessors',
 '_cast_all_from_to',
 '_comp',
 '_compare_to_non_df',
 '_compare_to_other_df',
 '_df',
 '_div',
 '_from_arrow',
 '_from_pandas',
 '_from_pydf',
 '_ipython_key_completions_',
 '_replace',
 '_repr_html_',
 '_row_encode',
 '_to_metadata',
 '_to_pandas_with_object_col

In [12]:
print(nodes_df.head())

shape: (5, 1)
┌─────────────────────────────────┐
│ halid	year	title	lang	domain	p… │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ 01907786	2018	Le Système Compt… │
│ 02029554	2018	Master Droit eur… │
│ 01724695	2017	Inversion method… │
│ 01730414	2017	split jacobians … │
│ 01735234	2018	Sociotechnique d… │
└─────────────────────────────────┘


In [19]:
dictio = nodes_df[0].to_dict()

In [23]:
dictio.values()

dict_values([shape: (1,)
Series: 'halid	year	title	lang	domain	paper_idx' [str]
[
	"01907786	2018	Le Système Compt…
]])

In [17]:
dir(nodes_df[0])

['__add__',
 '__annotations__',
 '__array__',
 '__arrow_c_stream__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_accessors',
 '_cast_all_from_to',
 '_comp',
 '_compare_to_non_df',
 '_compare_to_other_df',
 '_df',
 '_div',
 '_from_arrow',
 '_from_pandas',
 '_from_pydf',
 '_ipython_key_completions_',
 '_replace',
 '_repr_html_',
 '_row_encode',
 '_to_metadata',
 '_to_pandas_with_object_col

In [ ]:

# Create NetworkX graph
G = nx.Graph()

# Add nodes
for row in nodes_df.iter_rows(named=True):
    G.add_node(row['id'], **{k: v for k, v in row.items() if k != 'id'})

# Add edges
for row in edges_df.iter_rows(named=True):
    G.add_edge(row['source'], row['target'], **{k: v for k, v in row.items() if k not in ['source', 'target']})

print("Graph built with", G.number_of_nodes(), "nodes and", G.number_of_edges(), "edges.")


In [3]:
# Load the CSVs
#nodes_df = pd.read_csv(f"{data_path}/raw/nodes/papers.csv")
edges_df = pd.read_csv(f"{data_path}/raw/edges/paper__cites__paper.csv")

In [4]:
edges_df

,paper_idx\tc_paper_idx
0,0\t7705
1,0\t7706
2,0\t7707
3,0\t7721
4,0\t7728
...,...
27932966,7630\t23412
27932967,7669\t291146
27932968,7669\t291147
27932969,7615\t291148


In [2]:

# Create a graph (use nx.DiGraph() if it's directed)
G = nx.Graph()

# Add nodes (and optional attributes)
for _, row in nodes_df.iterrows():
    G.add_node(row['id'], label=row.get('label'))

# Add edges (with optional weight or other attributes)
for _, row in edges_df.iterrows():
    G.add_edge(row['source'], row['target'], weight=row.get('weight'))

# You can now use G!
print("Nodes:", G.nodes(data=True))
print("Edges:", G.edges(data=True))


ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2
